## 获取大模型

In [52]:
from itertools import chain

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from nbformat.v4 import output_from_msg

llm = ChatOpenAI(
    model="google/gemma-3n-e4b",
    openai_api_key="lm-studio",
    openai_api_base="http://localhost:1234/v1"
)

print(llm.invoke("10个字回答我：什么是大模型"))

APIConnectionError: Connection error.

# 使用提示词模版

chain = prompt | llm

chain: 这是一个 LangChain 中用于定义一系列步骤的链。它将 prompt 和 LLM 连接起来，形成一个完整的流程。
prompt | llm: 这使用了 LangChain 的 | 运算符（即 pipe 操作符）。它将 prompt 和 llm 连接起来，表示：
首先使用 prompt 模板创建 prompt。
然后将生成的 prompt 发送给 llm（大型语言模型，例如 OpenAI 的 GPT 模型）。
LLM 会根据 prompt 生成回复。
chain = ...: 将构建好的链赋值给变量 chain。

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是世界级的专家"),
        ("user","{input}")
    ])

chain = prompt | llm
message = chain.invoke({"input":"两句话，大模型的langchain是啥"})
print(message.content)

Langchain 是一个强大的开源框架，旨在简化大型语言模型 (LLM) 的应用开发，它提供了一系列工具和模块，用于连接 LLM 与其他数据源和工具，从而构建更复杂、更智能的应用。

简单来说，Langchain 就像一个 LLM 的“瑞士军刀”，帮助开发者更容易地利用 LLM 的能力，例如问答、文本生成、代码执行等，并将其集成到实际应用中。



# 使用输出解析器

In [36]:
from langchain_core.output_parsers import JsonOutputParser

output_parser =JsonOutputParser()

chain = prompt | llm | output_parser

message = chain.invoke({"input":"大模型的langchain是什么？，用JSON格式，5句话内回答，问题用question,回答用answer"})
print(message)


{'question': 'Langchain是什么？', 'answer': 'Langchain 是一个旨在简化大型语言模型 (LLM) 应用开发的框架。它提供了一系列工具，用于连接 LLM 与其他数据源和工具，构建复杂的链式应用。核心功能包括模型集成、提示管理、链（chains）和代理（agents）。它旨在提高 LLM 应用的可扩展性和实用性，降低开发难度。Langchain 促进了 LLM 在各种场景中的应用，例如问答、文本生成和数据分析。'}


In [ ]:
# 使用向量存储

In [68]:
# ===============================
# 1. 加载网页
# ===============================
from langchain_community.document_loaders import WebBaseLoader
import bs4

url = "https://www.gov.cn/zhengce/zhengceku/202504/content_7021191.htm"

loader = WebBaseLoader(
    web_path=url,
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
)
docs = loader.load()

print("网页加载完成：", len(docs), "个文档")


# ===============================
# 2. 文本分块
# ===============================
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = text_splitter.split_documents(docs)
texts = [d.page_content for d in documents]

print("文本分块完成：", len(texts), "块")


# ===============================
# 3. LM Studio Embedding（自定义类）
# ===============================
import requests
import numpy as np
from langchain_core.embeddings import Embeddings

class LMStudioEmbeddings(Embeddings):
    def __init__(self, base_url="http://localhost:1234/v1", model="text-embedding-nomic-embed-text-v1.5"):
        self.base_url = base_url
        self.model = model

    def embed_query(self, text: str):
        payload = {"input": text, "model": self.model}
        r = requests.post(f"{self.base_url}/embeddings", json=payload)
        return r.json()["data"][0]["embedding"]

    def embed_documents(self, texts):
        vectors = []
        for t in texts:
            vectors.append(self.embed_query(t))
        return vectors


embedder = LMStudioEmbeddings(
    base_url="http://localhost:1234/v1",
    model="text-embedding-nomic-embed-text-v1.5"
)

print("Embedding 模型准备完成")


# ===============================
# 4. 生成向量（逐条，不走 LangChain 的批处理）
# ===============================
vectors = embedder.embed_documents(texts)
vectors = np.array(vectors)

print("向量生成完成：", vectors.shape)


# ===============================
# 5. 使用 FAISS 构建向量库（正确格式）
# ===============================
from langchain_community.vectorstores import FAISS

text_embedding_pairs = list(zip(texts, vectors))

vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,   # ← 关键：传二元组列表
    embedding=embedder,
    metadatas=[{} for _ in texts],
)

print("向量库构建完成！")


# ===============================
# 6. 检索
# ===============================
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

query = "这个政策的主要内容是什么？"

# 新版本写法
relevant_docs = retriever.invoke(query)

context = "\n\n".join([d.page_content for d in relevant_docs])

print("检索完成：", len(relevant_docs), "条")


# ===============================
# 7. 使用 LM Studio LLM 回答（ChatOpenAI）
# ===============================
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="google/gemma-3n-e4b",
    openai_api_key="lm-studio",
    openai_api_base="http://localhost:1234/v1",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是专业政策分析助手，请基于给定的上下文回答问题，不允许编造内容。"),
    ("user", """
问题：{question}

上下文：
{context}

请总结回答。
""")
])

chain = prompt | llm

response = chain.invoke({
    "question": query,
    "context": context
})

print("\n================== 最终回答 ==================\n")
print(response.content)

网页加载完成： 1 个文档
文本分块完成： 5 块
Embedding 模型准备完成
向量生成完成： (5, 768)
向量库构建完成！
检索完成： 4 条

================== 最终回答 ==================

这个政策的主要内容是：**在2025年5月（第五个“民法典宣传月”）以“民法典进企业”为重点，开展一系列普法宣传活动，旨在提高企业诚信守法经营意识和能力，助力企业防范化解风险，促进经济高质量发展。**

具体来说，政策强调：

*   **以习近平法治思想为指导，深入学习宣传民法典。**
*   **重点开展“民法典进企业”活动，并结合其他领域（如机关、乡村、社区等）开展宣传。**
*   **突出学习宣传习近平法治思想，重点强调民法典的基本原则和相关规定。**
*   **加强与各部门的协调配合，形成广泛参与的普法工作格局。**
*   **鼓励创新宣传方式，运用传统和现代化手段相结合，提升宣传效果。**
*   **强调反对形式主义，注重服务企业，不给企业增加负担。**

总而言之，该政策旨在通过广泛的宣传活动，让民法典深入企业，提升企业法律意识，营造良好的法治环境。

